In [49]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials

In [50]:
from dotenv import load_dotenv
import os

load_dotenv()  # .envファイルを読み込む

KEY = os.environ.get("KEY")
ENDPOINT = os.environ.get("ENDPOINT")
# FaceAPIクライアントのインスタンス作成
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [51]:
# Person GroupのIDを指定
person_group_id = 'keisuke'

In [ ]:
# keisukeというグループを作成する
# face_client.person_group.create(
#     person_group_id,
#     name = "keisuke group"
# )

# グループ削除
# face_client.person_group.delete(person_group_id)

In [53]:
# person groupを確認
group_keisuke = face_client.person_group.get(person_group_id)
print(group_keisuke)


{'additional_properties': {}, 'name': 'keisuke group', 'user_data': None, 'recognition_model': None, 'person_group_id': 'keisuke'}


In [ ]:
# TashiroKeisukeの名前を登録
keisuke_name = 'Tashiro Keisuke'


In [ ]:
# keisuke_groupにkeisukeという名前のPersonを登録
# keisuke = face_client.person_group_person.create(
#     person_group_id = group_keisuke.person_group_id, # PersonGroupのIDを指定
#     name = keisuke_name # 登録するPersonの名前を指定
# )
# print(keisuke)

In [ ]:
# 画像送信でケイスケ写真をPersonに紐づける
with open('img/IMG_2857.JPG', 'rb') as image:
    keisuke_face_1 = face_client.person_group_person.add_face_from_stream(
        person_group_id = group_keisuke.person_group_id,
        person_id = keisuke.person_id,
        image = image
    )
with open('img/IMG_2859.JPG', 'rb') as image:
    keisuke_face_2 = face_client.person_group_person.add_face_from_stream(
        person_group_id = group_keisuke.person_group_id,
        person_id = keisuke.person_id,
        image = image
    )
with open('img/IMG_2860.JPG', 'rb') as image:
    keisuke_face_3 = face_client.person_group_person.add_face_from_stream(
        person_group_id = group_keisuke.person_group_id,
        person_id = keisuke.person_id,
        image = image
    )
with open('img/IMG_2861.JPG', 'rb') as image:
    keisuke_face_4 = face_client.person_group_person.add_face_from_stream(
        person_group_id = group_keisuke.person_group_id,
        person_id = keisuke.person_id,
        image = image
    )

# ケイスケPersonに登録した画像情報
print(keisuke_face_1)
print(keisuke_face_2)
print(keisuke_face_3)
print(keisuke_face_4)

In [ ]:
# PersonGroupPerson get で最新のPerson情報を取得する
# 上記で登録した画像idが含まれているはず
keisuke = face_client.person_group_person.get(
        person_group_id = group_keisuke.person_group_id,
        person_id = keisuke.person_id
    )
print(keisuke)

# Peasonの削除
# keisuke = face_client.person_group_person.delete(
#         person_group_id = group_keisuke.person_group_id,
#         person_id = keisuke.person_id
#     )
# print(keisuke)

In [ ]:
# 確認
group_keisuke = face_client.person_group.get(
    person_group_id,
    name = "keisuke group"
)
print(group_keisuke)
print(keisuke)

In [57]:
# 画像から表情、感情を判別 ケイスケではない写真
with open('img/IMG_2854.JPG', 'rb') as image:
    detected_faces = face_client.face.detect_with_stream(
        image, 
        return_face_attributes = ['age','gender','smile','emotion']
    )

# ケイスケの写真も読み込み
with open('img/IMG_2860.JPG', 'rb') as image:
    detected_faces_2 = face_client.face.detect_with_stream(
        image, 
        return_face_attributes = ['age','gender','smile','emotion']
    )

print(detected_faces[0].face_id)
print(detected_faces[0].face_attributes)
print(detected_faces[0].face_attributes.emotion)

5f6c6812-b64d-49ec-99ee-ce621c9b885f
{'additional_properties': {}, 'age': 33.0, 'gender': <Gender.male: 'male'>, 'smile': 0.997, 'facial_hair': None, 'glasses': None, 'head_pose': None, 'emotion': <azure.cognitiveservices.vision.face.models._models_py3.Emotion object at 0x7f9a3d2afee0>, 'hair': None, 'makeup': None, 'occlusion': None, 'accessories': None, 'blur': None, 'exposure': None, 'noise': None, 'mask': None, 'quality_for_recognition': None}
{'additional_properties': {}, 'anger': 0.0, 'contempt': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.997, 'neutral': 0.003, 'sadness': 0.0, 'surprise': 0.0}


In [ ]:
# personとfaceの比較
verified = face_client.face.verify_face_to_person(
    face_id = detected_faces[0].face_id,
    person_group_id = group_keisuke.person_group_id,
    person_id = keisuke.person_id
)
print(verified)

verified2 = face_client.face.verify_face_to_person(
    face_id = detected_faces_2[0].face_id,
    person_group_id = group_keisuke.person_group_id,
    person_id = keisuke.person_id
)
print(verified2)

if verified.confidence > 0.7:
    print("完全に本人")
elif verified.confidence > 0.5:
    print("ほぼ本人")
else:
    print("別人")

In [66]:
# 感情・表情を抽出
smile = format(detected_faces[0].face_attributes.smile * 100, '.1f')
anger = detected_faces[0].face_attributes.emotion.anger
contempt = detected_faces[0].face_attributes.emotion.contempt
disgust = detected_faces[0].face_attributes.emotion.disgust
fear = detected_faces[0].face_attributes.emotion.fear
happiness = detected_faces[0].face_attributes.emotion.happiness
neutral = detected_faces[0].face_attributes.emotion.neutral
sadness = detected_faces[0].face_attributes.emotion.sadness
surprise = detected_faces[0].face_attributes.emotion.surprise

print('笑顔(score:{0})\n怒り(score:{1})\n侮蔑(score:{2})\n嫌悪(score:{3})\n恐怖(score:{4})\n幸福(score:{5})\n自然(score:{6})\n悲哀(score:{7})\n驚き(score:{8})' \
.format(smile,anger,contempt,disgust,fear,happiness,neutral,sadness,surprise))

笑顔(score:99.70)
怒り(score:0.0)
侮蔑(score:0.0)
嫌悪(score:0.0)
恐怖(score:0.0)
幸福(score:0.997)
自然(score:0.003)
悲哀(score:0.0)
驚き(score:0.0)


In [63]:
a = "HELLO"
print('{0}\nWorld'.format(a))
# '{0}\n顔写真を送って!'
print('HELLOn\World')

HELLO
World
HELLOn\World
